In [1]:
import pandas as pd

# stats to calculate
- ## outsourcing:
  - percentage of books scanned at BPOs per year - done!
- ## pay
    - calculations on maximum wages workers at cebu could have made based on number of workers there per month and the contract IA had with Innodata at the time (at 1.5million per year / 12 months with x workers working per month, that leaves what max monthly wage).
- ## rates of work:
  - as measure of pages scanned to workers working per center per month
  - avergage pages scanned to workers working per center + standard deviation
- ## turn over rates- how long do operators stay at their jobs across centers
  - avg. days worked by workers at each center + standard deviation - will need to throw out lots of junk

In [2]:
# accessing the geocoded texts dataset from my external hard drive. You can get it from this box link: 
texts = pd.read_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/geocoded-texts-data.csv', low_memory=False)
texts = texts.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'])

In [7]:
# scans per center per month
scans_center_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/metadata-analysis/metadata-records-analysis-csvs/scans_per_center_per_month.csv") 

# pages per center per month
pages_center_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/metadata-analysis/metadata-records-analysis-csvs/pages_per_center_per_month.csv")

# scans per center type per month
scans_type_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/metadata-analysis/metadata-records-analysis-csvs/scans_per_center_type_per_month.csv").dropna(subset=['center_type'])
scans_type_month = scans_type_month[['month_year','center_type', 'books_scanned']]


In [8]:
scans_type_year = scans_type_month.groupby(['center_type','month_year']).sum().reset_index()

In [9]:
scans_type_year['year'] = ""

for i in range(len(scans_type_year)):
    scans_type_year.at[i, 'year'] = str(scans_type_year.at[i,'month_year']).split('-')[0]

In [10]:
scans_type_year = scans_type_year.groupby(['center_type','year']).sum().reset_index()[['center_type', 'year', 'books_scanned']]

In [11]:
scans_type_year

,center_type,year,books_scanned
0,academic,2001,3
1,academic,2002,421
2,academic,2006,37248
3,academic,2007,108590
4,academic,2008,164714
...,...,...,...
118,public,2019,21036
119,public,2020,8342
120,public,2021,16017
121,public,2022,29775


In [14]:
scans_per_year = scans_type_year.groupby(['year']).sum().reset_index()[['year','books_scanned']]
scans_per_year.loc[scans_per_year['year']== '2010']['books_scanned'].tolist()[0]

261573

In [8]:
scans_type_year['percent'] = ''

for i in range(len(scans_type_year)):
    scans_type_year.at[i, 'percent'] = scans_type_year.at[i, 'books_scanned'] / scans_per_year.loc[scans_per_year['year'] == scans_type_year.at[i, 'year']]['books_scanned'].tolist()[0]

 

In [9]:
scans_type_year.to_csv('/Users/elizabethschwartz/Documents/GitHub/ia_scanning_labor_data/metadata-analysis/metadata-records-analysis-csvs/percentage_of_books_scanned_per_center_type_per_year.csv')

In [10]:
scans_type_year

,center_type,year,books_scanned,percent
0,academic,2001,3,1.0
1,academic,2002,421,0.97907
2,academic,2006,37248,0.589554
3,academic,2007,108590,0.683085
4,academic,2008,164714,0.639723
...,...,...,...,...
118,public,2019,21036,0.043464
119,public,2020,8342,0.010275
120,public,2021,16017,0.016478
121,public,2022,29775,0.027982


In [13]:
scans_type_year['center_type'].tolist()
this_dict = {'year':"",'academic':"", 'bpo':"", 'government':"", 'hq':"", 'museum':"", 'public':"", 'total_books':""}
year_list = []
for i in range(20): 
    year_list.append(this_dict)

year_list

[{'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'mus

In [12]:
def get_center_count(center_type, year): 
    try:
        result = scans_type_year.loc[(scans_type_year['center_type'] == center_type) & (scans_type_year['year'] == year)].reset_index().to_dict()['books_scanned'][0]
    except KeyError: 
        result = 0
    return result


def get_center_percent(center_type, year):
    try:
        result = scans_type_year.loc[(scans_type_year['center_type'] == center_type) & (scans_type_year['year'] == year)].reset_index().to_dict()['percent'][0]
        result = f"{result:.2%}"
    except KeyError: 
        result = 0
    return result


In [13]:
get_center_count('academic', '1998')

0

In [27]:
the_years = scans_per_year['year'].tolist()
yearly_counts = []
for this_year in the_years: 
    yearly_counts.append({'year': this_year,
                         'academic': get_center_count('academic', this_year),
                         'bpo': get_center_count('bpo', this_year), 
                         'government': get_center_count('government', this_year), 
                         'hq': get_center_count('hq', this_year), 
                         'museum': get_center_count('museum', this_year),
                         'public': get_center_count('public', this_year), 
                         'total_books':""})

yearly_percents = []
for this_year in the_years: 
    yearly_percents.append({'year': this_year,
                         'academic': get_center_percent('academic', this_year),
                         'bpo': get_center_percent('bpo', this_year), 
                         'government': get_center_percent('government', this_year), 
                         'hq': get_center_percent('hq', this_year), 
                         'museum': get_center_percent('museum', this_year),
                         'public': get_center_percent('public', this_year)})
  

In [152]:
the_years = scans_per_year['year'].tolist()

In [16]:
scans_type_year.loc[(scans_type_year['center_type'] == 'academic') & (scans_type_year['year'] == '2012')]

,center_type,year,books_scanned,percent
8,academic,2012,91679,0.250427


In [28]:
yearly_percents

[{'year': '2001',
  'academic': '100.00%',
  'bpo': 0,
  'government': 0,
  'hq': 0,
  'museum': 0,
  'public': 0},
 {'year': '2002',
  'academic': '97.91%',
  'bpo': 0,
  'government': '0.93%',
  'hq': 0,
  'museum': '0.23%',
  'public': '0.93%'},
 {'year': '2006',
  'academic': '58.96%',
  'bpo': 0,
  'government': 0,
  'hq': '41.04%',
  'museum': '0.00%',
  'public': 0},
 {'year': '2007',
  'academic': '68.31%',
  'bpo': 0,
  'government': '0.25%',
  'hq': '28.38%',
  'museum': '0.57%',
  'public': '2.49%'},
 {'year': '2008',
  'academic': '63.97%',
  'bpo': 0,
  'government': '9.77%',
  'hq': '12.01%',
  'museum': '0.59%',
  'public': '13.66%'},
 {'year': '2009',
  'academic': '56.18%',
  'bpo': '0.00%',
  'government': '17.74%',
  'hq': '9.40%',
  'museum': '0.59%',
  'public': '15.96%'},
 {'year': '2010',
  'academic': '51.05%',
  'bpo': '2.18%',
  'government': '9.80%',
  'hq': '22.61%',
  'museum': '0.30%',
  'public': '13.93%'},
 {'year': '2011',
  'academic': '31.95%',
  'bpo

In [19]:


yearly_counts_df = pd.DataFrame.from_records(yearly_counts) 
yearly_counts_df


,year,academic,bpo,government,hq,museum,public,total_books
0,2001,3,0,0,0,0,0,
1,2002,421,0,4,0,1,4,
2,2006,37248,0,0,25931,1,0,
3,2007,108590,0,402,45116,899,3963,
4,2008,164714,0,25163,30925,1510,35165,
5,2009,135579,11,42811,22681,1414,38515,
6,2010,133532,5714,25634,59152,784,36441,
7,2011,98462,130463,30265,25,465,44831,
8,2012,91679,188778,35093,685,282,39508,
9,2013,89153,20919,48684,64,1819,36198,


In [31]:
yearly_percents_df = pd.DataFrame.from_records(yearly_percents) 

In [32]:
yearly_percents_df

,year,academic,bpo,government,hq,museum,public
0,2001,100.00%,0,0,0,0,0
1,2002,97.91%,0,0.93%,0,0.23%,0.93%
2,2006,58.96%,0,0,41.04%,0.00%,0
3,2007,68.31%,0,0.25%,28.38%,0.57%,2.49%
4,2008,63.97%,0,9.77%,12.01%,0.59%,13.66%
5,2009,56.18%,0.00%,17.74%,9.40%,0.59%,15.96%
6,2010,51.05%,2.18%,9.80%,22.61%,0.30%,13.93%
7,2011,31.95%,42.34%,9.82%,0.01%,0.15%,14.55%
8,2012,25.04%,51.57%,9.59%,0.19%,0.08%,10.79%
9,2013,45.14%,10.59%,24.65%,0.03%,0.92%,18.33%


In [33]:
yearly_counts_df.to_csv('/Users/elizabethschwartz/Documents/GitHub/ia_scanning_labor_data/metadata-analysis/metadata-records-analysis-csvs/count_books_scanned_per_year_per_type.csv')
yearly_percents_df.to_csv('/Users/elizabethschwartz/Documents/GitHub/ia_scanning_labor_data/metadata-analysis/metadata-records-analysis-csvs/percentage_books_scanned_per_year_per_type.csv')


## overall percentages

In [47]:
scans_type_year

total_scans_by_type =  scans_type_year.groupby(['center_type']).sum().reset_index()[['center_type','books_scanned']]

In [48]:
total_scans_by_type

,center_type,books_scanned
0,academic,1314866
1,archive,21134
2,bpo,4048959
3,government,700435
4,hq,185807
5,museum,35502
6,public,472975


In [51]:
total_scans_by_type['percent']= total_scans_by_type['books_scanned']/total_scans_by_type['books_scanned'].sum() * 100

In [52]:
total_scans_by_type

,center_type,books_scanned,percent
0,academic,1314866,19.394225
1,archive,21134,0.311726
2,bpo,4048959,59.721996
3,government,700435,10.331390
4,hq,185807,2.740646
5,museum,35502,0.523653
6,public,472975,6.976364


In [35]:
total_scans_by_type

,center_type,books_scanned,percent
0,academic,1314866,19
1,archive,21134,0.
2,bpo,4048959,59
3,government,700435,10
4,hq,185807,2.
5,museum,35502,0.
6,public,472975,6.
